In [20]:
import csv
import logging
import requests
from bs4 import BeautifulSoup
import time
import re
import json
logging.basicConfig(format='%(message)s')

In [21]:
def normalize_query(q):
    q = q.replace('\n', '')
    q = q.replace('（', '(')
    q = q.replace('）', ')')
    q = re.sub(r"\(.+\)$", "", q)
    q = re.sub('(!|\u3000|/|\\s|>|<|\\.)+', " ", q)
    return q

def search(q):
    url_search = 'https://eiga.com/search/{}' .format(requests.utils.quote(normalize_query(q), safe=''))
    res_search = requests.get(url_search )
    res_search.encoding = res_search.apparent_encoding
    
    soup_search = BeautifulSoup(res_search.content, "lxml")
    result =  soup_search.find('section', attrs={"id": "rslt-movie"})
    if result != None:
        path = result.find('li', attrs={"class": "col-s-3"}).find('a')["href"]
        return path
    else:
        return None

def scrape(query, movie_id, year):
    data = {
        "id": -1,
        "rating": -1,
        "check-in":-1,
        "review-count":-1
    }
    print("START : " + query)
    path = search(query)
    
    if path is None:
        logging.warning("**************************************************")
        logging.warning(query + " HAS NO RESULT")
        logging.warning("**************************************************")
        with open('./eigacom_id_table_{}_error.txt'.format(year), 'a') as f:
            print(str(movie_id), "-1")
            f.write(str(movie_id) + '\t' + '-1' + '\n')
        return None
    
    url_review = 'https://eiga.com' + path
    eigacom_id = re.sub("\\D", "", path)
    
    with open('./eigacom_id_table_{}.txt'.format(year), 'a') as f:
        print(str(movie_id), eigacom_id)
        f.write(str(movie_id) + '\t' + eigacom_id + '\n')
    
    res = requests.get(url_review)
    res.encoding = res.apparent_encoding
    soup = BeautifulSoup(res.content, "lxml")
    
    rating = soup.find('span', attrs={"class": "rating-star"})
    review_count = soup.find('span', attrs={"itemprop": "reviewCount"})
    check_in = soup.find('a', attrs={"class": "icon-movie-checkin"}).find('strong')
    data["rating"] = 0 if rating.text == '－' else float(rating.text)
    data["review-count"] = 0 if review_count is None else int(review_count.text)
    data["check-in"] = 0 if check_in is None else int(check_in.text)
        
    return data

In [22]:
def main():
    years = ['2017', '2016', '2015', '2014']
    for y in years:
        print(y)
        fr = open('./eigacom_id_table_{}.txt'.format(y), 'r')
        with open( '../{}_movie_clean'.format(y), 'r') as movie_clean:
            for line in csv.reader(movie_clean, delimiter='\t'):
                eigacom_id = fr.readline().split()[1]
                if eigacom_id != "-1":
                    continue
                    
                movie_id, title, *_ = line
            
                output_file = './{0}/{1}.json'.format(y, movie_id)
                with open(output_file, 'w') as f:
                    print(movie_id)
                    data = scrape(title,movie_id, y)
                    if data == None:
                        continue
                    data["id"] = int(movie_id)
                    json.dump(data, f, ensure_ascii=False, indent=2) 
                time.sleep(1)
    fr.close()
    

In [23]:
if __name__ == '__main__':
    main()

2017
2
START : 傷物語III 冷血篇
2 83227
10
START : 劇場版 動物戦隊ジュウオウジャーVSニンニンジャー 未来からのメッセージ from スーパー戦隊
10 86069
11
START : 劇場版 新・ミナミの帝王
11 84729
18
START : 劇場版 黒執事 Book of the Atlantic
18 83283
22
START : イタズラなkiss THE MOVIE 2 〜キャンバス編〜


**************************************************
イタズラなkiss THE MOVIE 2 〜キャンバス編〜 HAS NO RESULT
**************************************************


22 -1
24
START : 劇場版 BiS誕生の詩
24 86317
40
START : 劇場版 ソードアート・オンライン -オーディナル・スケール-
40 83224
58
START : 劇場版プリパラ み～んなでかがやけ！キラリン☆スターライブ！
58 85902
61
START : 劇場版 しまじろうのわお！しまじろうと にじのオアシス


**************************************************
劇場版 しまじろうのわお！しまじろうと にじのオアシス HAS NO RESULT
**************************************************


61 -1
63
START : 劇場版 ウルトラマンオーブ 絆の力、おかりします！
63 86049
64
START : 劇場版総集編 オーバーロード 漆黒の英雄
64 86054
70
START : 劇場版 黒子のバスケ LAST GAME
70 84708
75
START : サクラダリセット 前篇
75 85654
99
START : 劇場版 Free!-Timeless Medley- 絆
99 86805
111
START : 劇場版 FAIRY TAIL -DRAGON CRY-
111 86574
116
START : サクラダリセット 後篇
116 85655
139
START : 22年目の告白 -私が殺人犯です-
139 85298
147
START : 劇場版 魔法科高校の劣等生 星を呼ぶ少女
147 84507
153
START : 宇宙戦艦ヤマト2202 愛の戦士たち/第二章 発進篇
153 86606
166
START : 劇場版ポケットモンスター キミにきめた!
166 86155
186
START : 劇場版 仮面ライダーエグゼイド トゥルー・エンディング
186 86299
203
START : 劇場版Fate/kaleid liner プリズマ☆イリヤ 雪下の誓い
203 86604
204
START : 映画監督外山文治短編作品集


...

663 -1
670
START : 夜だから Night, Because


**************************************************
夜だから Night, Because HAS NO RESULT
**************************************************


670 -1
755
START : さよなら東京


**************************************************
さよなら東京 HAS NO RESULT
**************************************************


755 -1


映画.comにないやつ<br>
2017 <br>
204 外山文治短編作品集<br>
2016 <br>
169 葛城物語<br>
2014 <br>
326 SiM「10 YEARS - SPECIAL EDiTiON -」
351 アンダーフェイス
390 MOOSIC LAB 2014/あんこまん
461 新・極道の紋章
663 0.5ミリ
755 さよなら東京
2013 <br>
78 英国ロイヤル・バレエ/ロミオとジュリエット
83 つるしびな
165 英国ロイヤル・バレエ/不思議の国のアリス
404 イン・ザ・シャドウズ
477 ブシモ Presentsプロレス3D第7弾 ワールドプロレスリング G1 CLIMAX 2013

In [29]:
errors_2017 = [
    (83967, 22),
    (83446,59),
    (85425, 239),
    (84639, 248),
    (86831, 292),
]

errors_2016 = [
    (82315, 32),
    (85928,61),
    (83806, 175),
]

errors_2015 = [
    (81351, 44),
    (81588,131),
    (78671, 166),
]

errors_2014 = [
    (79824, 107),
    (79222,141),
    (79672, 186),
    (77705, 190),
    (79960,320),
    (79998, 342),
    (80145, 348),
    (80146,391),
    (80421, 410),
    (80420, 411),
    (80147,413),
    (80419, 416),
    (79957, 427),
    (79260,483),
    (78987, 500),
    (79842, 569),
    (80979,654),
    (81192, 670),
]

errors_2013 = [
    (78009,100),
    (78140, 103),
    (58247, 111),
    (77852, 261),
    (78088, 294),
    (77941, 350),
    (78149, 432),
    (79303, 516),
    (79202, 633),
]

In [30]:
errors_all = [errors_2017, errors_2016, errors_2015, errors_2014, errors_2013]

In [31]:
def scrape_directly(id_in, id_out):
    page_num = 1
    data = {
        "id": id_out,
        "rating": -1,
        "check-in":-1,
        "review-count":-1
    }
    url_review = "https://eiga.com/movie/" + str(id_in)
    
    if(url_review == "error"):
        return "error"
    

    res = requests.get(url_review)
    res.encoding = res.apparent_encoding
    soup = BeautifulSoup(res.content, "lxml")
    
    rating = soup.find('span', attrs={"class": "rating-star"})
    review_count = soup.find('span', attrs={"itemprop": "reviewCount"})
    check_in = soup.find('a', attrs={"class": "icon-movie-checkin"}).find('strong')
    data["rating"] = 0 if rating.text == '－' else float(rating.text)
    data["review-count"] = 0 if review_count is None else int(review_count.text)
    data["check-in"] = 0 if check_in is None else int(check_in.text)
    print('DONE : ' + str(id_out) )
    return data

In [32]:
year = 2017
for errors in errors_all:
    print(year)
    for id_in, id_out in errors:
        output_file = './{0}/{1}.json'.format(str(year), id_out)
        with open(output_file, 'w') as f:
            data = scrape_directly(id_in, id_out)
            jsn =  json.dump(data,f,ensure_ascii=False, indent=2) 
        time.sleep(1)
    year -= 1

2017
DONE : 22
DONE : 59
DONE : 239
DONE : 248
DONE : 292
2016
DONE : 32
DONE : 61
DONE : 175
2015
DONE : 44
DONE : 131
DONE : 166
2014
DONE : 107
DONE : 141
DONE : 186
DONE : 190
DONE : 320
DONE : 342
DONE : 348
DONE : 391
DONE : 410
DONE : 411
DONE : 413
DONE : 416
DONE : 427
DONE : 483
DONE : 500
DONE : 569
DONE : 654
DONE : 670
2013
DONE : 100
DONE : 103
DONE : 111
DONE : 261
DONE : 294
DONE : 350
DONE : 432
DONE : 516
DONE : 633
